In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from selenium import webdriver
from urllib.request import urlopen
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [14]:
url = 'https://baoquangninh.com.vn/search?keyword=c%C3%BAm+gia+c%E1%BA%A7m&keyword_extra=&cate=&filter=title&datestart=01-01-2000&dateend=01-08-2021&chinhxac=on&search='
root = 'https://baoquangninh.com.vn'
df = pd.DataFrame(columns = ['Title', 'URL', 'Date', 'Content'])

In [15]:
for i in range(2, 25):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    news = soup.find_all('div', class_ = 'col-lg-6 mb-3')
    for item in news:
        title = item.find('h3', class_ = 'card-title')['title']
        sub_url = item.find('a')['href']
        date = item.find('time', class_ = 'card-time').text
        date = datetime.strptime(date.strip().split('| ')[1], '%d/%m/%Y')
        if sub_url.startswith('https'):
            url = sub_url
        else:
            url = root + sub_url
        response = requests.get(url).text
        new_soup = BeautifulSoup(response, 'lxml')
        content = new_soup.find('div', class_ = 'detail auto-gallery font3 mt-3')
        if content:
            content = content.text
        new_row = {'Title': title, 'URL': url, 'Date': date, 'Content': content}
        df = df.append(new_row, ignore_index = True)
    url = 'https://baoquangninh.com.vn/search?keyword=c%C3%BAm+gia+c%E1%BA%A7m&keyword_extra=&cate=&filter=title&datestart=01-01-2000&dateend=01-08-2021&chinhxac=on&search=&page=' + str(i)

In [16]:
df

,Title,URL,Date,Content
0,UBND tỉnh chỉ đạo khẩn trương triển khai các b...,https://baoquangninh.com.vn/ubnd-tinh-chi-dao-...,2021-07-11,"\nNgày 9/7, UBND tỉnh ban hành Công văn số 437..."
1,"Phòng, chống cúm gia cầm A/H5N8 lần đầu tiên x...",https://baoquangninh.com.vn/phong-chong-cum-gi...,2021-07-06,\nCác địa phương sẽ sử dụng các loại vaccine c...
2,Quảng Ninh xuất hiện chủng cúm gia cầm mới H5N8,https://baoquangninh.com.vn/quang-ninh-xuat-hi...,2021-07-03,None
3,Cảnh giác cao và chủ động ngăn chặn vi rút cúm...,https://baoquangninh.com.vn/canh-giac-cao-va-c...,2021-07-03,\nTheo Chi cục Chăn nuôi và Thú y tỉnh (Sở NN&...
4,Quảng Ninh xuất hiện chủng cúm gia cầm lần đầu...,https://baoquangninh.com.vn/quang-ninh-xuat-hi...,2021-07-02,"\nTheo tin từ Chi cục Chăn nuôi và Thú y tỉnh,..."
...,...,...,...,...
225,Tái phát dịch cúm gia cầm,https://baoquangninh.com.vn/tai-phat-dich-cum-...,2006-12-20,\nCục Thú y - Bộ Nông nghiệp & Phát triển Nông...
226,Sẽ không có dịch cúm gia cầm trong mùa đông - ...,https://baoquangninh.com.vn/se-khong-co-dich-c...,2006-12-13,\nMùa đông năm nay liệu có tái phát dịch cúm g...
227,Nguy cơ cao tái phát dịch cúm gia cầm,https://baoquangninh.com.vn/nguy-co-cao-tai-ph...,2006-08-24,"\nTại cuộc họp Ban Chỉ đạo quốc gia phòng, chố..."
228,Triển khai tiêm vắc xin phòng dịch cúm gia cầm,https://baoquangninh.com.vn/trien-khai-tiem-va...,2006-08-18,"\nTừ ngày 10-9 đến 10-10, toàn tỉnh Quảng Ninh..."


In [17]:
df.to_csv('data/baoquangninh_articles_vietnamese.csv', index = False)